# 🤖 Notebook 04: RAG Pipeline

Build a Retrieval-Augmented Generation pipeline that:
1. Retrieves relevant product reviews via FAISS
2. Re-ranks by rating + sentiment
3. Generates natural language answers grounded in real reviews

Supports OpenAI API (if key set) or local extractive summary.

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath('..'))
os.environ.setdefault('SAMPLE_ONLY', 'true')

from src.config import Config
from src.data_ingest import load_flipkart, get_product_names
from src.embedding_model import EmbeddingModel
from src.indexer import FAISSIndexer
from src.retriever import DenseRetriever
from src.reranker import Reranker
from src.rag_pipeline import RAGPipeline
from src.utils import load_pickle, save_pickle

cfg = Config()

In [ ]:
# ── Load data and build pipeline ───────────────────────────────────
df = load_flipkart(cfg)
texts = df['combined_text'].tolist()
metadata = df.to_dict('records')

emb = EmbeddingModel(cfg)
try:
    vectors = load_pickle(cfg.DATA_PROCESSED / 'embeddings.pkl')
except FileNotFoundError:
    vectors = emb.encode(texts, normalize=True)

indexer = FAISSIndexer(dim=emb.dim, index_type='hnsw', cfg=cfg)
indexer.add(vectors)

retriever = DenseRetriever(indexer, emb, texts, metadata)
reranker = Reranker(rating_weight=0.15, sentiment_weight=0.10)
rag = RAGPipeline(retriever, reranker, top_k=10, rerank_k=5)
print(f'✅ RAG Pipeline ready — {len(texts)} docs indexed')

In [ ]:
# ── RAG: Question Answering ────────────────────────────────────────
questions = [
    'Which product has the best battery life?',
    'Are there any common complaints about the air cooler?',
    'What do customers say about the smartwatch build quality?',
    'Which product offers the best value for money?',
    'Are there any issues with product delivery?',
]

for q in questions:
    print('\n' + '='*70)
    print(f'🗣️ QUESTION: {q}')
    print('='*70)
    answer = rag.answer(q)
    print(answer)
    print('-'*70)

In [ ]:
# ── RAG with Product Filter ────────────────────────────────────────
products = get_product_names(df, cfg)
if products:
    target_product = products[0][:20]  # Use first product
    print(f'🏷️ Product-filtered RAG for: "{target_product}"\n')
    answer = rag.answer('What are the main pros and cons?', product_filter=target_product)
    print(answer)

In [ ]:
# ── Show retrieved context (transparency) ──────────────────────────
query = 'good quality air cooler'
results = rag.retrieve(query)
print(f'📋 Retrieved context for: "{query}"\n')
for r in results:
    product = str(r.metadata.get('product_name', ''))[:30]
    rating = r.metadata.get('Rating', '?')
    sentiment = r.metadata.get('Sentiment', '?')
    print(f'  [{product}] Rating: {rating}/5 ({sentiment})')
    print(f'  > {r.text[:150]}')
    print()

### Summary
- RAG pipeline retrieves relevant product reviews and synthesizes grounded answers
- Re-ranker boosts high-rated, positive-sentiment reviews to the top
- Product filtering enables product-specific Q&A
- Local fallback works without any API key — shows source reviews with ratings
- When OpenAI key is provided, enables full natural language generation